这是一个基于PyTorch实现的DSFD（Dual Shot Face Detector）人脸检测模型训练脚本，专门针对暗光场景进行了优化。该实现采用了双阶段检测架构，结合了图像增强网络（RetinexNet）和检测网络，以提高模型在低光照条件下的性能。代码支持多种基础网络（VGG、ResNet系列和专门的暗光场景模型），并实现了完整的分布式训练流程，包括数据加载、模型训练、验证和模型保存等功能。在训练过程中，使用了多任务损失函数（检测损失和图像增强损失），并采用SSIM（结构相似性）评估图像质量。代码还实现了动态学习率调整、梯度裁剪、权重衰减等优化策略，并支持断点续训和TensorBoard可视化。通过结合图像增强和检测网络，该实现显著提高了模型在暗光环境下的检测性能，适合在WIDER FACE等大规模数据集上进行训练

核心库导入

In [2]:
# %load train.py

from __future__ import division
from __future__ import absolute_import
from __future__ import print_function

# ===================== 导入必要的库 =====================
# 基础库
import os
import random
import time
import torch
import argparse
import torch.optim as optim
import torch.utils.data as data
import numpy as np
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchmetrics.functional import structural_similarity_index_measure as ssim
import torchvision.utils as vutils
from torch.utils.tensorboard import SummaryWriter


In [ ]:


from data.config import cfg  # 配置文件，包含模型参数和训练设置
from layers.modules import MultiBoxLoss, EnhanceLoss  # 损失函数模块
from data.widerface import WIDERDetection, detection_collate  # 数据集加载和预处理
from models.factory import build_net, basenet_factory  # 模型构建工厂
from models.enhancer import RetinexNet  # 图像增强网络
from utils.DarkISP import Low_Illumination_Degrading  # 低光照图像生成工具
from PIL import Image


这个部分定义了训练过程中所有可配置的参数，包括模型选择、训练批次大小、学习率等关键超参数

In [ ]:


parser = argparse.ArgumentParser(
    description='DSFD face Detector Training With Pytorch')
train_set = parser.add_mutually_exclusive_group()
# 训练相关参数
parser.add_argument('--batch_size',
                    default=4, type=int,
                    help='训练批次大小，影响内存使用和训练速度')
parser.add_argument('--model',
                    default='dark', type=str,
                    choices=['dark', 'vgg', 'resnet50', 'resnet101', 'resnet152'],
                    help='选择训练模型，dark为暗光场景专用模型')
parser.add_argument('--resume',
                    default=None, type=str,
                    help='恢复训练的检查点文件路径，用于断点续训')
parser.add_argument('--num_workers',
                    default=0, type=int,
                    help='数据加载的工作进程数，建议设置为CPU核心数的2-4倍')
parser.add_argument('--cuda',
                    default=True, type=bool,
                    help='是否使用CUDA进行训练，建议在有GPU的情况下开启')
parser.add_argument('--lr', '--learning-rate',
                    default=5e-4, type=float,
                    help='初始学习率，影响模型收敛速度和稳定性')
parser.add_argument('--momentum',
                    default=0.9, type=float,
                    help='SGD优化器的动量值，用于加速收敛和减少震荡')
parser.add_argument('--weight_decay',
                    default=5e-4, type=float,
                    help='SGD优化器的权重衰减，用于防止过拟合')
parser.add_argument('--gamma',
                    default=0.1, type=float,
                    help='SGD学习率更新的gamma值，控制学习率衰减速度')
parser.add_argument('--multigpu',
                    default=True, type=bool,
                    help='是否使用多GPU训练，可以加速训练过程')
parser.add_argument('--save_folder',
                    default='weights/',
                    help='保存检查点模型的目录，用于存储训练过程中的模型')
parser.add_argument('--local_rank',
                    type=int,
                    help='分布式训练的本地rank，用于多GPU训练时的进程标识')

args = parser.parse_args()
global local_rank
local_rank = args.local_rank


这个部分设置了分布式训练的环境，对于多GPU训练至关重要，确保数据正确分配到各个GPU；以及对数据加载器设置，这个部分负责数据的加载和预处理，使用分布式采样器确保数据在多GPU训练时正确分配

In [ ]:


if 'LOCAL_RANK' not in os.environ:
    os.environ['LOCAL_RANK'] = str(args.local_rank)

# 初始化分布式进程组
import torch.distributed as dist
dist.init_process_group(backend='nccl')


if torch.cuda.is_available():
    if args.cuda:
        gpu_num = torch.cuda.device_count()
        if local_rank == 0:
            print('使用 {} 个GPU'.format(gpu_num))
        rank = int(os.environ['RANK'])
        torch.cuda.set_device(rank % gpu_num)
    if not args.cuda:
        print("警告: 您有CUDA设备，但未使用CUDA。\n使用 --cuda 参数以获得最佳训练速度。")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')


save_folder = os.path.join(args.save_folder, args.model)
if not os.path.exists(save_folder):
    os.mkdir(save_folder)


train_dataset = WIDERDetection(cfg.FACE.TRAIN_FILE, mode='train')
val_dataset = WIDERDetection(cfg.FACE.VAL_FILE, mode='val')

# 创建分布式采样器
train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset, shuffle=True)
train_loader = data.DataLoader(train_dataset, args.batch_size,
                               num_workers=args.num_workers,
                               collate_fn=detection_collate,
                               sampler=train_sampler,
                               pin_memory=True)

val_batchsize = args.batch_size
val_sampler = torch.utils.data.distributed.DistributedSampler(val_dataset, shuffle=True)
val_loader = data.DataLoader(val_dataset, val_batchsize,
                             num_workers=0,
                             collate_fn=detection_collate,
                             sampler=val_sampler,
                             pin_memory=True)

min_loss = np.inf


训练函数是整个训练脚本的核心部分，负责模型的初始化、训练循环、验证和模型保存等关键功能。在训练过程中，首先初始化模型和优化器，加载预训练权重或恢复训练状态。训练循环中，对输入图像进行预处理和增强，包括生成暗光图像和使用增强网络处理图像。在前向传播阶段，模型同时处理暗光图像和正常图像，计算检测损失和增强损失。损失函数包括第一阶段的定位损失和置信度损失、第二阶段的定位损失和置信度损失，以及图像增强相关的重建损失和SSIM损失。在反向传播阶段，使用梯度裁剪防止梯度爆炸，并通过优化器更新模型参数。训练过程中定期进行验证，保存最佳模型和检查点，并使用TensorBoard记录训练指标和可视化结果。整个训练过程支持分布式训练，可以充分利用多GPU资源加速训练。

In [ ]:

def train():
    """
    主训练函数
    包含模型初始化、训练循环、验证和模型保存等功能
    
    训练流程：
    1. 初始化模型和优化器
    2. 加载预训练权重或恢复训练
    3. 训练循环：
       - 数据预处理和增强
       - 前向传播
       - 损失计算
       - 反向传播和优化
       - 定期验证和保存
    4. 模型评估和保存
    """
    # 初始化TensorBoard写入器
    writer = None

    # 计算每个epoch的迭代次数
    per_epoch_size = len(train_dataset) // (args.batch_size * torch.cuda.device_count())
    start_epoch = 0
    iteration = 0
    step_index = 0

    # 创建TensorBoard日志目录
    log_dir = os.path.join('runs', args.model, time.strftime('%Y%m%d-%H%M%S'))
    if local_rank == 0 and not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # 初始化TensorBoard
    if local_rank == 0:
        writer = SummaryWriter(log_dir=log_dir)
        print(f"TensorBoard日志将保存到: {log_dir}")

    # ===================== 模型初始化 =====================
    # 创建基础网络和DSFD网络
    basenet = basenet_factory(args.model)  # 创建基础网络（如VGG、ResNet等）
    dsfd_net = build_net('train', cfg.NUM_CLASSES, args.model)  # 创建DSFD检测网络
    net = dsfd_net
    net_enh = RetinexNet()  # 创建图像增强网络
    net_enh.load_state_dict(torch.load('/data1/home/chenruoyu/DAI-Net/weights/decomp.pth'))  # 加载预训练的增强网络权重

    # 加载预训练权重或恢复训练
    if args.resume:
        if local_rank == 0:
            print('恢复训练，加载 {}...'.format(args.resume))
        start_epoch = net.load_weights(args.resume)
        iteration = start_epoch * per_epoch_size
    else:
        base_weights = torch.load(args.save_folder + basenet)
        if local_rank == 0:
            print('加载基础网络 {}'.format(args.save_folder + basenet))
        if args.model == 'vgg' or args.model == 'dark':
            net.vgg.load_state_dict(base_weights)  # 加载VGG或DarkNet基础网络权重
        else:
            net.resnet.load_state_dict(base_weights)  # 加载ResNet基础网络权重

    # 初始化网络权重
    if not args.resume:
        if local_rank == 0:
            print('初始化权重...')
        # 初始化各个模块的权重
        net.extras.apply(net.weights_init)  # 特征提取模块
        net.fpn_topdown.apply(net.weights_init)  # FPN自顶向下路径
        net.fpn_latlayer.apply(net.weights_init)  # FPN横向连接
        net.fpn_fem.apply(net.weights_init)  # 特征增强模块
        net.loc_pal1.apply(net.weights_init)  # 第一阶段定位预测
        net.conf_pal1.apply(net.weights_init)  # 第一阶段置信度预测
        net.loc_pal2.apply(net.weights_init)  # 第二阶段定位预测
        net.conf_pal2.apply(net.weights_init)  # 第二阶段置信度预测
        net.ref.apply(net.weights_init)  # 特征细化模块

    # ===================== 优化器设置 =====================
    # 设置学习率，根据batch size和GPU数量进行缩放
    lr = args.lr * np.round(np.sqrt(args.batch_size / 4 * torch.cuda.device_count()), 4)
    param_group = []
    # 为不同模块设置不同的学习率
    param_group += [{'params': dsfd_net.vgg.parameters(), 'lr': lr}]  # 基础网络
    param_group += [{'params': dsfd_net.extras.parameters(), 'lr': lr}]  # 特征提取
    param_group += [{'params': dsfd_net.fpn_topdown.parameters(), 'lr': lr}]  # FPN自顶向下
    param_group += [{'params': dsfd_net.fpn_latlayer.parameters(), 'lr': lr}]  # FPN横向连接
    param_group += [{'params': dsfd_net.fpn_fem.parameters(), 'lr': lr}]  # 特征增强
    param_group += [{'params': dsfd_net.loc_pal1.parameters(), 'lr': lr}]  # 第一阶段定位
    param_group += [{'params': dsfd_net.conf_pal1.parameters(), 'lr': lr}]  # 第一阶段置信度
    param_group += [{'params': dsfd_net.loc_pal2.parameters(), 'lr': lr}]  # 第二阶段定位
    param_group += [{'params': dsfd_net.conf_pal2.parameters(), 'lr': lr}]  # 第二阶段置信度
    param_group += [{'params': dsfd_net.ref.parameters(), 'lr': lr / 10.}]  # 特征细化（使用较小的学习率）

    optimizer = optim.SGD(param_group, lr=lr, momentum=args.momentum,
                          weight_decay=args.weight_decay)

    # ===================== GPU和分布式设置 =====================
    if args.cuda:
        if args.multigpu:
            # 使用DistributedDataParallel进行多GPU训练
            net = torch.nn.parallel.DistributedDataParallel(net.cuda(), find_unused_parameters=True)
            net_enh = torch.nn.parallel.DistributedDataParallel(net_enh.cuda())
        cudnn.benchmark = True  # 启用cuDNN自动调优

    # 初始化损失函数
    criterion = MultiBoxLoss(cfg, args.cuda)  # 目标检测损失
    criterion_enhance = EnhanceLoss()  # 图像增强损失
    if local_rank == 0:
        print('加载WIDER数据集...')
        print('使用指定的参数:')
        print(args)

    # 调整学习率
    for step in cfg.LR_STEPS:
        if iteration > step:
            step_index += 1
            adjust_learning_rate(optimizer, args.gamma, step_index)

    # ===================== 训练循环 =====================
    net_enh.eval()  # 设置增强网络为评估模式
    net.train()  # 设置检测网络为训练模式
    for epoch in range(start_epoch, cfg.EPOCHES):
        losses = 0

        for batch_idx, (images, targets, _) in enumerate(train_loader):
            # 数据预处理
            images = Variable(images.cuda() / 255.)  # 归一化图像
            targetss = [Variable(ann.cuda(), requires_grad=False) for ann in targets]  # 处理目标框
            # 生成暗光图像
            img_dark = torch.empty(size=(images.shape[0], images.shape[1], images.shape[2], images.shape[3])).cuda()
            for i in range(images.shape[0]):
                img_dark[i], _ = Low_Illumination_Degrading(images[i])

            # 学习率调整
            if iteration in cfg.LR_STEPS:
                step_index += 1
                adjust_learning_rate(optimizer, args.gamma, step_index)

            # 前向传播
            t0 = time.time()
            # 使用增强网络处理图像
            R_dark_gt, I_dark = net_enh(img_dark)  # 获取暗光图像的反射图和光照图
            R_light_gt, I_light = net_enh(images)  # 获取正常图像的反射图和光照图

            # 使用检测网络进行预测
            out, out2, loss_mutual = net(img_dark, images, I_dark.detach(), I_light.detach())
            R_dark, R_light, R_dark_2, R_light_2 = out2

            # 计算损失
            optimizer.zero_grad()
            # 计算检测损失
            loss_l_pa1l, loss_c_pal1 = criterion(out[:3], targetss)  # 第一阶段损失
            loss_l_pa12, loss_c_pal2 = criterion(out[3:], targetss)  # 第二阶段损失
            # 计算增强损失
            loss_enhance = criterion_enhance([R_dark, R_light, R_dark_2, R_light_2, I_dark.detach(), I_light.detach()], images, img_dark) * 0.1
            # 计算重建损失和SSIM损失
            loss_enhance2 = F.l1_loss(R_dark, R_dark_gt.detach()) + F.l1_loss(R_light, R_light_gt.detach()) + (
                        1. - ssim(R_dark, R_dark_gt.detach())) + (1. - ssim(R_light, R_light_gt.detach()))

            # 总损失
            loss = loss_l_pa1l + loss_c_pal1 + loss_l_pa12 + loss_c_pal2 + loss_enhance2 + loss_enhance + loss_mutual
            
            # 反向传播和优化
            loss.backward()
            # 梯度裁剪，防止梯度爆炸
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=35, norm_type=2)
            optimizer.step()
            t1 = time.time()
            losses += loss.item()

            # 打印训练信息
            if iteration % 100 == 0:
                tloss = losses / (batch_idx + 1)
                if local_rank == 0:
                    print('计时器: %.4f' % (t1 - t0))
                    print('epoch:' + repr(epoch) + ' || iter:' +
                          repr(iteration) + ' || Loss:%.4f' % (tloss))
                    print('->> pal1 置信度损失:{:.4f} || pal1 定位损失:{:.4f}'.format(
                        loss_c_pal1.item(), loss_l_pa1l.item()))
                    print('->> pal2 置信度损失:{:.4f} || pal2 定位损失:{:.4f}'.format(
                        loss_c_pal2.item(), loss_l_pa12.item()))
                    print('->>学习率:{}'.format(optimizer.param_groups[0]['lr']))

                    # TensorBoard可视化
                    if iteration % 500 == 0:
                        original_img = images[0].cpu()
                        dark_img = img_dark[0].cpu()
                        enhanced_img = R_dark[0].detach().cpu()

                        writer.add_image('Images/原始图像', original_img, iteration)
                        writer.add_image('Images/暗光图像', dark_img, iteration)
                        writer.add_image('Images/增强图像', enhanced_img, iteration)

            # 保存检查点
            if iteration != 0 and iteration % 5000 == 0:
                if local_rank == 0:
                    print('保存状态, iter:', iteration)
                    file = 'dsfd_' + repr(iteration) + '.pth'
                    torch.save(dsfd_net.state_dict(),
                               os.path.join(save_folder, file))
            iteration += 1

        # 验证
        if (epoch + 1) >= 0:
            val(epoch, net, dsfd_net, net_enh, criterion, writer)
        if iteration >= cfg.MAX_STEPS:
            break



验证函数负责在验证集上评估模型的性能，是监控训练进度和保存最佳模型的关键组件。在验证过程中，首先将模型设置为评估模式，然后在验证集上进行前向传播。对于每个批次的数据，函数会生成对应的暗光图像，并通过模型进行预测。验证过程中计算第一阶段的定位损失和置信度损失，以及第二阶段的定位损失和置信度损失，这些损失共同构成总验证损失。函数会汇总所有GPU上的损失，计算平均验证损失，并通过TensorBoard记录验证结果。如果当前验证损失低于历史最低损失，函数会保存当前模型作为最佳模型。此外，函数还会定期保存检查点，包含当前训练轮次和模型权重，以便后续恢复训练。

In [ ]:

def val(epoch, net, dsfd_net, net_enh, criterion, writer=None):
    """
    验证函数
    在验证集上评估模型性能
    
    验证流程：
    1. 设置模型为评估模式
    2. 在验证集上进行前向传播
    3. 计算验证损失
    4. 保存最佳模型
    5. 记录验证结果
    """
    net.eval()  # 设置模型为评估模式
    step = 0
    losses = torch.tensor(0.).cuda()
    t1 = time.time()

    # 验证循环
    for batch_idx, (images, targets, img_paths) in enumerate(val_loader):
        if args.cuda:
            images = Variable(images.cuda() / 255.)
            targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
        else:
            images = Variable(images / 255.)
            targets = [Variable(ann, volatile=True) for ann in targets]
        
        # 生成暗光图像
        img_dark = torch.stack([Low_Illumination_Degrading(images[i])[0] for i in range(images.shape[0])], dim=0)
        # 前向传播
        out, R = net.module.test_forward(img_dark)

        # 计算损失
        loss_l_pa1l, loss_c_pal1 = criterion(out[:3], targets)  # 第一阶段损失
        loss_l_pa12, loss_c_pal2 = criterion(out[3:], targets)  # 第二阶段损失
        loss = loss_l_pa12 + loss_c_pal2  # 总损失

        losses += loss.item()
        step += 1
    
    # 汇总所有GPU的损失
    dist.reduce(losses, 0, op=dist.ReduceOp.SUM)

    # 计算平均损失
    tloss = losses / step / torch.cuda.device_count()
    t2 = time.time()
    
    # 打印验证结果
    if local_rank == 0:
        print('计时器: %.4f' % (t2 - t1))
        print('验证 epoch:' + repr(epoch) + ' || Loss:%.4f' % (tloss))

        # TensorBoard记录
        if writer is not None:
             writer.add_scalar('Loss/验证损失', tloss, epoch)

             # 验证集图像可视化
             if epoch % 1 == 0:
                 original_val_img = images[0].cpu()
                 enhanced_val_img = R[0].detach().cpu()

                 writer.add_image('验证图像/原始图像', original_val_img, epoch)
                 writer.add_image('验证图像/增强图像', enhanced_val_img, epoch)

    # 保存最佳模型
    global min_loss
    if tloss < min_loss:
        if local_rank == 0:
            print('保存最佳模型, epoch', epoch)
            torch.save(dsfd_net.state_dict(), os.path.join(save_folder, 'dsfd.pth'))
        min_loss = tloss

    # 保存检查点
    states = {
        'epoch': epoch,
        'weight': dsfd_net.state_dict(),
    }
    if local_rank == 0:
        torch.save(states, os.path.join(save_folder, 'dsfd_checkpoint.pth'))


该函数接收三个关键参数：优化器对象、学习率衰减系数（gamma）和当前训练步骤。在训练过程中，当达到预定义的步骤时，函数会被调用，将优化器中所有参数组的学习率乘以衰减系数gamma，实现学习率的动态调整。

In [ ]:

def adjust_learning_rate(optimizer, gamma, step):
    """
    调整学习率
    在每个指定步骤将学习率衰减gamma倍
    
    参数:
        optimizer: 优化器对象
        gamma: 学习率衰减系数
        step: 当前步骤
    """
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr'] * gamma

# ===================== 主函数 =====================
if __name__ == '__main__':
    train()